<a href="https://colab.research.google.com/github/lucken99/ConstitutionXpert/blob/main/ProjectCI_Aug26.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
## Install all required libraries
!pip install langchain
!pip install pypdf

!pip install pdfminer.six
# !pip install unstructured pdf2image  # for unstructured pdf loader (legacy)

!pip install tiktoken
!pip install sentence_transformers
!pip install chromadb
!pip install cohere
!pip install openai
!pip install litellm
!pip install rank_bm25
!pip install google-api-python-client
# !pip install ipython
# !pip install IPykernel
!pip install nest-asyncio
!pip install html2text


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 272.3/272.3 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 67.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 77.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 115.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 85.4 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-

# DATA

> We have text file which consists of paragraphs related to Indian Constitution for e.g., Articles, Schedules, etc.

>

In [2]:
# data path
dir_path = "/content/drive/MyDrive/Project_CILLM/db/"
text_file_path = "/content/drive/MyDrive/Project_CILLM/db/file_context_corpus_cleaned_extended_part3.txt"
text_file_path = "/content/drive/MyDrive/Project_CILLM/colab_project/Constitution-Xpert using OpenAI Embeddings (1)/file_context_corpus_cleaned_extended_part3.txt"
pdf_file_path = "/content/drive/MyDrive/Project_CILLM/db/file_context_corpus_cleaned_extended_part3.pdf"


In [3]:
# utility function to load api keys from json file
import json
path_to_keys = "/content/drive/MyDrive/Project_CILLM/Keys/keys.json"
def return_api_key(name):
    with open(path_to_keys, 'r') as f:
        json_data = json.load(f)
        return json_data[name]

# RETRIEVAL


## [Document loaders](https://python.langchain.com/docs/modules/data_connection/document_loaders/)

In [4]:
# Text Loaders
from langchain.document_loaders import TextLoader

# PDF Loaders (try which suits us best)
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import PyPDFDirectoryLoader
# from langchain.document_loaders import MathpixPDFLoader
from langchain.document_loaders import UnstructuredPDFLoader
from langchain.document_loaders import PDFMinerPDFasHTMLLoader
from langchain.document_loaders import AmazonTextractPDFLoader
from langchain.document_loaders import OnlinePDFLoader

loader_text = TextLoader(text_file_path)
loader_pdf = PyPDFLoader(pdf_file_path)

In [5]:
## utility function for document loaders information

def loaded_doc_info(loader, show_loaded_data=False):
    data = loader.load()
    print("Type of the loader:", type(loader))
    print("Length of the data:", len(data))
    if show_loaded_data:
        print(data)
    return data




In [6]:
# loaded_doc_info(loader_text, show_loaded_data=True)
text_data = loaded_doc_info(loader_text)

Type of the loader: <class 'langchain.document_loaders.text.TextLoader'>
Length of the data: 1


In [7]:
pdf_data = loaded_doc_info(loader_pdf)

Type of the loader: <class 'langchain.document_loaders.pdf.PyPDFLoader'>
Length of the data: 178


In [ ]:
# # checking different pdf loader
# loader = PDFMinerPDFasHTMLLoader(pdf_file_path)
# data = loaded_doc_info(loader, True)

## [Document Transformers](https://python.langchain.com/docs/modules/data_connection/document_transformers/)

In [8]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.text_splitter import MarkdownHeaderTextSplitter  # we can split by parts in constitution using this splitter and add metadata for better search
                                                                # first we have to add headers (for e.g., # or ##)

# # Split by tokens
# from langchain.text_splitter import TokenTextSplitter
# from langchain.text_splitter import SpacyTextSplitter
# from langchain.text_splitter import NLTKTextSplitter

# !pip install tiktoken
# good for OpenAI Models
# text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
#     chunk_size=100, chunk_overlap=0
# )

# # Sentence Transformers token split
# from langchain.text_splitter import SentenceTransformersTokenTextSplitter # for a particular sentence transformer

# # Hugging face tokenizers
# from transformers import GPT2TokenizerFast
# tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")
# text_splitter = CharacterTextSplitter.from_huggingface_tokenizer(
#     tokenizer, chunk_size=100, chunk_overlap=0
# )
# texts = text_splitter.split_text(text)


In [9]:
def split_data(loader, splitter):
    docs = loader.load_and_split(splitter)
    print("Loader Type:", type(loader))
    print("Splitter Type:", type(splitter))
    print("Length of splitted data:", len(docs))
    return docs




In [ ]:
# using openai's tiktoken for splitting data
text_splitter = CharacterTextSplitter.from_tiktoken_encoder()
docs_text = split_data(loader_text, text_splitter)
docs_pdf = split_data(loader_pdf, text_splitter)

Loader Type: <class 'langchain.document_loaders.text.TextLoader'>
Splitter Type: <class 'langchain.text_splitter.CharacterTextSplitter'>
Length of splitted data: 31
Loader Type: <class 'langchain.document_loaders.pdf.PyPDFLoader'>
Splitter Type: <class 'langchain.text_splitter.CharacterTextSplitter'>
Length of splitted data: 178


In [ ]:
# using RecursiveCharacterTextSplitter
recur_splitter = RecursiveCharacterTextSplitter()
docs_text = split_data(loader_text, recur_splitter)
docs_pdf = split_data(loader_pdf, recur_splitter)

Loader Type: <class 'langchain.document_loaders.text.TextLoader'>
Splitter Type: <class 'langchain.text_splitter.RecursiveCharacterTextSplitter'>
Length of splitted data: 156
Loader Type: <class 'langchain.document_loaders.pdf.PyPDFLoader'>
Splitter Type: <class 'langchain.text_splitter.RecursiveCharacterTextSplitter'>
Length of splitted data: 178


In [ ]:
docs_pdf[0]

In [10]:
char_splitter = CharacterTextSplitter(
    separator = "\n\n",
    # chunk_size = 1000,
    # chunk_overlap  = 200,
    # length_function = len,
    is_separator_regex = False,
)

# char_splitter = CharacterTextSplitter(chunk_size=2000, chunk_overlap=500)

docs_text = split_data(loader_text, char_splitter)
docs_pdf = split_data(loader_pdf, char_splitter)


Loader Type: <class 'langchain.document_loaders.text.TextLoader'>
Splitter Type: <class 'langchain.text_splitter.CharacterTextSplitter'>
Length of splitted data: 156
Loader Type: <class 'langchain.document_loaders.pdf.PyPDFLoader'>
Splitter Type: <class 'langchain.text_splitter.CharacterTextSplitter'>
Length of splitted data: 178


In [11]:
docs = [doc.page_content for doc in docs_text]

In [ ]:
# docs_pdf[0].page_content

In [ ]:
# print(docs_pdf[0].page_content.rstrip('\n'))

## Text Embedding models and chromadb Vectorstore
[MTEB blog](https://huggingface.co/blog/mteb) <br>
[MTEB](https://huggingface.co/spaces/mteb/leaderboard)

### Chromadb VectorStore


In [12]:
from langchain.vectorstores import Chroma


In [13]:
# utility function for embeddings info

def embeddings_info(embeddings):
    print("Total Embeddings:", len(embeddings))
    print("Dimension:", len(embeddings[0]))


In [14]:
### Problems with Chroma
### https://python.langchain.com/docs/integrations/vectorstores/chroma#basic-example-including-saving-to-disk
## Caching Embeddings using LocalFileStore
# from langchain.storage import LocalFileStore
# from langchain.embeddings import CacheBackedEmbeddings # Embeddings can be stored or temporarily cached to avoid needing to recompute them.
# import os

# utility function for creating embeddings using chromdb
# def create_cached_embeddings(docs, embedding_model, model_name, fs):
#     # print(list(fs.yield_keys()))
#     # cached_embedder = CacheBackedEmbeddings.from_bytes_store(
#     #     embedding_model, fs, namespace=model_name
#     # )
#     ### Create the vectorstore
#     # db = Chroma.from_documents(docs, cached_embedder)
#     db = Chroma.from_documents(docs, embedding_model)
#     return db

# def create_embeddings(docs, embedding_model):
#     return Chroma.from_documents(docs, embedding_model)


# utility function for similarity search
def return_similar_docs(db, query, k=4, show_docs=False):
    docs = db.similarity_search_with_relevance_scores(query, k=k)
    docs = sorted(docs, key=lambda x: -x[1])
    if show_docs:
        for doc in docs:
            print("Text:", doc[0].page_content)
            print("Relevance Score:", doc[1])
            print("--"*50)
            print("--"*50)
    return docs



### BGE Hugging face embeddings

In [16]:
from langchain.embeddings import HuggingFaceBgeEmbeddings

model_name = "BAAI/bge-large-en"
model_kwargs = {'device': 'cuda'}
encode_kwargs = {'normalize_embeddings': True}
hf = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

In [ ]:
# embeddings = hf.embed_documents(docs)
# embeddings_info(embeddings)
# sentence1 = "Tell me about article 23"
# embedding1 = hf.embed_query(sentence1)
# import numpy as np

# dot_product = np.dot(embeddings, embedding1)
# dot_product
# np.argmax()

In [18]:
hf_db = Chroma.from_documents(docs_text, hf, persist_directory="./chroma_db_hf")
# hf_db = Chroma(persist_directory="./chroma_db_hf", embedding_function=hf)

In [19]:
### check similar documents
query = "What is writs in constitution?"
retrived_docs = return_similar_docs(hf_db, query, show_docs=False)

### Cohere Embeddings

In [20]:
COHERE_API_KEY = return_api_key("COHERE_API_KEY")

In [21]:
from langchain.embeddings import CohereEmbeddings
cohr = CohereEmbeddings(
    model="embed-english-v2.0",
    cohere_api_key=COHERE_API_KEY
    )


In [22]:
# db_cohr = create_embeddings(docs_text, cohr)
cohr_db = Chroma.from_documents(docs_text, cohr, persist_directory="./chroma_db_cohr")
# cohr_db = Chroma(persist_directory="./chroma_db_cohr", embedding_function=cohr)

In [23]:
### check similar documents
query = "What is writs in constitution?"
retrieved_docs = return_similar_docs(cohr_db, query, show_docs=False)

/usr/local/lib/python3.10/dist-packages/langchain/vectorstores/base.py:254: UserWarning: Relevance scores must be between 0 and 1, got [(Document(page_content="Article 241: High Courts' power to issue certain writs\n(1) The High Court shall have power, throughout the territories in relation to which it exercises jurisdiction, to issue to any person or authority, including in appropriate cases, any Government, directions, orders or writs, including writs in the nature of habeas corpus, mandamus, prohibition, quo warranto and certiorari, or any of them, for the enforcement of any of the rights conferred by Part III and for any other purpose.\n(2) The power conferred by clause (1) shall not be in derogation of the power conferred on the Supreme Court by Article 32.\n\nArticle 242: Administration of tribal areas\n(1) The President may, with the consent of the Parliament, by order, direct that any area in the States of Assam, Meghalaya, Tripura and Mizoram shall be administered as a tribal 

#### Reordering
- When models must access relevant information in the middle of long contexts, then tend to ignore the provided documents. See: https://arxiv.org/abs/2307.03172


In [24]:
from langchain.document_transformers import (
    LongContextReorder,
)

# Reorder returned docs
reordering = LongContextReorder()
reordered_docs = reordering.transform_documents(retrieved_docs)
reordered_docs

[(Document(page_content='Government of India Act 1935:\nAn all-India Federation was proposed which would consist of British India and the princely states. This never materialised though. Subjects were divided between the centre and the provinces. Centre was in charge of the Federal List, provinces in charge of the Provincial List and there was a Concurrent List which both catered to. Diarchy was abolished at the provincial level and introduced at the centre. More autonomy was accorded to the provinces and in 6 out of 11 provinces, the bicameral legislature was introduced. A federal court was established and the Indian Council abolished. Burma and Aden were severed off from India. This act provided for the establishment of the RBI. This Act continued until it was replaced by the new Indian Constitution.\n\nIndian Independence Act 1947:\nIndia was declared independent and sovereign. The Viceroy and the Governors were made constitutional (nominal) heads. Set up responsible governments at 

### OpenAI Embeddings

In [ ]:
OPENAI_API_KEY = return_api_key("OPENAI_API_KEY")
# OPENAI_API_KEY

In [ ]:
from langchain.embeddings import OpenAIEmbeddings

openai = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)


#### We can also use [VectorstoreIndexCreator](https://python.langchain.com/docs/modules/data_connection/retrievers/#one-line-index-creation)
for creating vectorstore quickly with one liner code.

In [25]:
from langchain.indexes import VectorstoreIndexCreator
from langchain.chains import RetrievalQA


index_creator = VectorstoreIndexCreator(
    vectorstore_cls=Chroma,
    embedding=CohereEmbeddings(cohere_api_key=COHERE_API_KEY),
    text_splitter=CharacterTextSplitter(separator='\n\n'),
)

index = index_creator.from_loaders([loader_text])

# querying
query = "what is article 4?"
# index.query(query) # give your llm
# index.query_with_sources(query) # give your llm

# check Vectorstore
index.vectorstore

# can use as a retriever
index.vectorstore.as_retriever()

# can use in QA using llm
# qa = RetrievalQA.from_chain_type(llm=OpenAI(), chain_type="stuff", retriever=retriever)

VectorStoreRetriever(tags=['Chroma', 'CohereEmbeddings'], metadata=None, vectorstore=<langchain.vectorstores.chroma.Chroma object at 0x7e55a1d6fdf0>, search_type='similarity', search_kwargs={})

## [Retrievers](https://python.langchain.com/docs/modules/data_connection/retrievers/)

### [MultiQueryRetriever](https://python.langchain.com/docs/modules/data_connection/retrievers/MultiQueryRetriever)

In [26]:
# using cohere llm via LiteLLM lib interface
# from langchain.chat_models import ChatLiteLLM
from langchain.llms import Cohere
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import AIMessage, HumanMessage, SystemMessage
from langchain.retrievers.multi_query import MultiQueryRetriever

# llm = ChatLiteLLM(model="command-nightly", cohere_api_key=COHERE_API_KEY)
llm = Cohere(model="command-nightly", cohere_api_key=COHERE_API_KEY)
# llm.cohere_api_key
retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever=cohr_db.as_retriever(), llm=llm
)

In [27]:
# Set logging for the queries
import logging

logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)


In [28]:

question = "Tell me about article 4"
unique_docs = retriever_from_llm.get_relevant_documents(query=question)
len(unique_docs)

INFO:langchain.retrievers.multi_query:Generated queries: ['1. "Tell me about article 4 in relation to the others."', '2. "What is the main idea of article 4?"', '3. "Article 4 is about _________"']


9

- We can supply our own prompt along with [output parser](https://python.langchain.com/docs/modules/model_io/output_parsers/) to split the results into a list of queries



In [29]:
from typing import List
from langchain import LLMChain
from pydantic import BaseModel, Field
from langchain.prompts import PromptTemplate
from langchain.output_parsers import PydanticOutputParser


# Output parser will split the LLM result into a list of queries
class LineList(BaseModel):
    # "lines" is the key (attribute name) of the parsed output
    lines: List[str] = Field(description="Lines of text")


class LineListOutputParser(PydanticOutputParser):
    def __init__(self) -> None:
        super().__init__(pydantic_object=LineList)

    def parse(self, text: str) -> LineList:
        lines = text.strip().split("\n")
        return LineList(lines=lines)


output_parser = LineListOutputParser()

QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI language model assistant. Your task is to generate five
    different versions of the given user question to retrieve relevant documents from a vector
    database. By generating multiple perspectives on the user question, your goal is to help
    the user overcome some of the limitations of the distance-based similarity search.
    Provide these alternative questions seperated by newlines.
    Original question: {question}""",
)

# Chain
llm_chain = LLMChain(llm=llm, prompt=QUERY_PROMPT, output_parser=output_parser)

# Other inputs
question = "what is reservation in contitution of india?"

In [30]:
# Run
retriever = MultiQueryRetriever(
    retriever=cohr_db.as_retriever(), llm_chain=llm_chain, parser_key="lines"
)  # "lines" is the key (attribute name) of the parsed output

# Results
unique_docs = retriever.get_relevant_documents(
    query=question
)
len(unique_docs)

INFO:langchain.retrievers.multi_query:Generated queries: ['1. What is the reservation in the Constitution of India according to Article 15?', '2. How does the reservation system in India work?', '3. What are the different types of reservations in India?', '4. What is the history of reservation in India?', '5. How has the reservation system changed over time in India?']


10

### [Ensemble Retriever](https://python.langchain.com/docs/modules/data_connection/retrievers/ensemble)
- The EnsembleRetriever takes a list of retrievers as input and ensemble the results of their get_relevant_documents() methods and rerank the results based on the [Reciprocal Rank Fusion](https://plg.uwaterloo.ca/~gvcormac/cormacksigir09-rrf.pdf) algorithm.
- The most common pattern is to combine a sparse retriever(like BM25) with a dense retriever(like Embedding similarity), because their strengths are complementary. It is also known as "hybrid search".

In [31]:
from langchain.retrievers import BM25Retriever
from langchain.retrievers import EnsembleRetriever

In [32]:
# initialize the bm25 retriever
bm25_retriever = BM25Retriever.from_documents(docs_text)
bm25_retriever.k = 2

# initialize the Chromadb as retriever
chroma_retriever = cohr_db.as_retriever(search_kwargs={"k": 2})



In [33]:
# initialize the ensemble retriever
ensemble_retriever = EnsembleRetriever(retrievers=[bm25_retriever, chroma_retriever], weights=[0.5, 0.5])

In [34]:
docs = ensemble_retriever.get_relevant_documents("schedules")
docs

[Document(page_content='How many Schedules are there in India?\nThere are 12 Schedules in the Indian Constitution.\n\nWhat is the 11th and 12th Schedule of India?\nEleventh Schedule – It contains the provisions that specify the powers, authority and responsibilities of Panchayats.\nFundamental Rights – It deals with the provisions that specify the powers, authority and responsibilities of Municipalities.\n\nWhich act gave Constitutional recognition to the municipalities?\n74th Amendment Act 1992 gave constitutional recognition to urban local bodies i.e. municipalities.\n\nRepresentation of People Act, 1951:\nThe Representation of People Act, 1951 was passed by the Parliament in accordance with Article 327 of the Constitution of India. This legislation deals with the conduct of elections in the country and is hence a very important one. Articles 324 to 329 of Part XV of the Indian Constitution provides for the country’s electoral system. The Constitution confers upon the Parliament the 

### [MultiVector Retriever](https://python.langchain.com/docs/modules/data_connection/retrievers/multi_vector)

- The methods to create multiple vectors per document include:

    - smaller chunks: split a document into smaller chunks, and embed those (this is ParentDocumentRetriever)
    - summary: create a summary for each document, embed that along with (or instead of) the document
    - hypothetical questions: create hypothetical questions that each document would be appropriate to answer, embed those along with (or instead of) the document

In [35]:
from langchain.retrievers.multi_vector import MultiVectorRetriever
COHERE_API_KEY = return_api_key("COHERE_API_KEY")

In [36]:
from langchain.vectorstores import Chroma
from langchain.embeddings import CohereEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.storage import InMemoryStore
from langchain.document_loaders import TextLoader

In [37]:
loaders = TextLoader(text_file_path)

docs = loaders.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000)
docs = text_splitter.split_documents(docs)

#### Smaller chunks


In [38]:
# The vectorstore to use to index the child chunks
vectorstore = Chroma(
    collection_name="full_docs_CI",
    embedding_function=CohereEmbeddings(cohere_api_key=COHERE_API_KEY),
)

# The Storage layer for the parent documents
store = InMemoryStore()
id_key = "doc_id"

# The retriever (empty to start)
retriever = MultiVectorRetriever(
    vectorstore=vectorstore,
    docstore=store,
    id_key=id_key,
)

import uuid
doc_ids = [str(uuid.uuid4()) for _ in docs]


In [39]:
# The splitter to use to create the smaller chunks
child_text_splitter = RecursiveCharacterTextSplitter(chunk_size=800)

In [40]:
sub_docs = []
for i, doc in enumerate(docs):
    _id = doc_ids[i]
    _sub_docs = child_text_splitter.split_documents([doc])
    for _doc in _sub_docs:
        _doc.metadata[id_key] = _id
    sub_docs.extend(_sub_docs)

In [41]:
retriever.vectorstore.add_documents(sub_docs)
retriever.docstore.mset(list(zip(doc_ids, docs)))

In [42]:
# Vectorstore alone retrieves the small chunks
retriever.vectorstore.similarity_search("article 29")

[Document(page_content='Article 49: Protection of monuments and places and objects of national importance.\nArticle 50: Separation of judiciary from the executive.\nArticle 51: Promotion of international peace and security.', metadata={'doc_id': '79a8d0f9-f087-4bad-bcb0-fbe589b175cc', 'source': '/content/drive/MyDrive/Project_CILLM/colab_project/Constitution-Xpert using OpenAI Embeddings (1)/file_context_corpus_cleaned_extended_part3.txt'}),
 Document(page_content='Article 289: Exemption of property and income of a State from Union taxation.\nArticle 290: Adjustment in respect of certain expenses and pensions.\nArticle 290A: Annual payment to certain Devaswom Funds.\nArticle 291: [Repealed].', metadata={'doc_id': 'cf31db4e-34a4-4ad1-b5c1-9f748b7f221b', 'source': '/content/drive/MyDrive/Project_CILLM/colab_project/Constitution-Xpert using OpenAI Embeddings (1)/file_context_corpus_cleaned_extended_part3.txt'}),
 Document(page_content='Article 39: Certain principles of policy to be follow

In [43]:
# Retriever returns larger chunks
len(retriever.get_relevant_documents("article 29")[0].page_content)

8326

In [ ]:
# sub_docs

In [ ]:
# # cohr_db.similarity_search("article 29")
# bm25_retriever = BM25Retriever.from_documents(docs)
# bm25_retriever.k = 2

# ensemble_retriever = EnsembleRetriever(retrievers=[bm25_retriever, retriever], weights=[0.5, 0.5])
# ensemble_retriever.get_relevant_documents("article 29")

#### Summary

- Oftentimes a summary may be able to distill more accurately what a chunk is about, leading to better retrieval. Here we show how to create summaries, and then embed those.

In [44]:
from langchain.llms import Cohere
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
import uuid
from langchain.schema.document import Document

In [ ]:
chain = (
    {"doc": lambda x: x.page_content}
    | ChatPromptTemplate.from_template("Summarize the following document:\n\n{doc}")
    | Cohere(model="command-nightly", max_retries=0, cohere_api_key=COHERE_API_KEY)
    | StrOutputParser()
)

In [ ]:
# due to free license we are requesting only 5 API calls per minute , therefore this cell will take time
# import time
# # summaries = chain.batch(docs, {"max_concurrency": 1})
# summaries = []
# for i in range(len(docs)):
#     summaries.append(chain.invoke(doc))
#     if i%5 == 0:
#         time.sleep(60)

In [ ]:
# summary_docs = [Document(page_content=s,metadata={id_key: doc_ids[i]}) for i, s in enumerate(summaries)]

In [45]:
import pickle
# pickle.dump(summary_docs, open("summaries.txt", "wb"))
summary_docs = pickle.load(open("/content/drive/MyDrive/Project_CILLM/summaries.txt", 'rb'))

In [ ]:
summary_docs

In [47]:
retriever.vectorstore.add_documents(summary_docs)
retriever.docstore.mset(list(zip(doc_ids, docs)))

In [48]:
sub_docs = vectorstore.similarity_search("Tell me about Article 29")

In [49]:
sub_docs[0]

Document(page_content='Article 289: Exemption of property and income of a State from Union taxation.\nArticle 290: Adjustment in respect of certain expenses and pensions.\nArticle 290A: Annual payment to certain Devaswom Funds.\nArticle 291: [Repealed].', metadata={'doc_id': 'cf31db4e-34a4-4ad1-b5c1-9f748b7f221b', 'source': '/content/drive/MyDrive/Project_CILLM/colab_project/Constitution-Xpert using OpenAI Embeddings (1)/file_context_corpus_cleaned_extended_part3.txt'})

In [50]:
retrieved_docs = retriever.get_relevant_documents("Tell me bout Article 29")

[]

### [Self Querying](https://python.langchain.com/docs/modules/data_connection/retrievers/self_query/)

- A self-querying retriever is one that, as the name suggests, has the ability to query itself.
-  Specifically, given any natural language query, the retriever uses a query-constructing LLM chain to write a structured query and then applies that structured query to it's underlying VectorStore.
- This allows the retriever to not only use the user-input query for semantic similarity comparison with the contents of stored documented, but to also extract filters from the user query on the metadata of stored documents and to execute those filters.

### [WebResearchRetriever](https://python.langchain.com/docs/modules/data_connection/retrievers/web_research)

Given a query, this retriever will:

- Formulate a set of relate Google searches
- Search for each
- Load all the resulting URLs
- Then embed and perform similarity search with the query on the consolidate page content

In [51]:
from langchain.retrievers.web_research import WebResearchRetriever
GOOGLE_API_KEY = return_api_key("GOOGLE_API_KEY")
GOOGLE_CSE_ID = return_api_key("GOOGLE_CSE_ID")
COHERE_API_KEY = return_api_key("COHERE_API_KEY")


In [62]:
from langchain.vectorstores import Chroma
from langchain.embeddings import CohereEmbeddings
from langchain.utilities import GoogleSearchAPIWrapper
from langchain.llms import Cohere
from langchain.chat_models import ChatLiteLLM
embedding_function = CohereEmbeddings(cohere_api_key=COHERE_API_KEY)

# LLM

llm = Cohere(model="command-nightly", cohere_api_key=COHERE_API_KEY)
# bug in LiteLLM cohere integration
# use
import os
os.environ['COHERE_API_KEY'] = COHERE_API_KEY
llm = ChatLiteLLM(model="command-nightly", cohere_api_key=COHERE_API_KEY, )


In [63]:
# Vectorstore
vectorstore = Chroma(embedding_function=embedding_function, persist_directory="./chroma_db_cohr_go")



In [64]:
# Search
search = GoogleSearchAPIWrapper(google_api_key=GOOGLE_API_KEY, google_cse_id=GOOGLE_CSE_ID)

In [65]:
# Initialize
web_research_retriever = WebResearchRetriever.from_llm(
    vectorstore=vectorstore,
    llm=llm,
    search=search

)

In [67]:
import nest_asyncio
nest_asyncio.apply()

## Run with citations
from langchain.chains import RetrievalQAWithSourcesChain

user_input = "Explain article 29 in detail!"

qa_chain = RetrievalQAWithSourcesChain.from_chain_type(
    llm,
    retriever=web_research_retriever,
    max_tokens_limit=4096
)

result = qa_chain({"question": user_input})
result

Fetching pages: 100%|##########| 1/1 [00:00<00:00,  3.41it/s]


{'question': 'Explain article 29 in detail!',
 'answer': ' The Universal Declaration of Human Rights, or UDHR, is a document that was written by the United Nations in 1948. It is a declaration of all human rights and fundamental freedoms. It is not a binding document, but it does provide a basis for all human rights treaties. The document contains 30 articles that detail the rights of all human beings.',
 'sources': ''}

In [68]:
# Run with logging
import logging
logging.basicConfig()
logging.getLogger("langchain.retrievers.web_research").setLevel(logging.INFO)
user_input = "Explain article 29 of indian constitution in detail "
docs = web_research_retriever.get_relevant_documents(user_input)

INFO:langchain.retrievers.web_research:Generating questions for Google Search ...
Your text contains a trailing whitespace, which has been trimmed to ensure high quality generations.
INFO:langchain.retrievers.web_research:Questions for Google Search (raw): {'question': 'Explain article 29 of indian constitution in detail ', 'text': LineList(lines=['1. "What is article 29 of the indian constitution about ?"\n', '2. "article 29 of indian constitution"\n', '3. "Explain article 29 of indian constitution in detail ?"'])}
INFO:langchain.retrievers.web_research:Questions for Google Search: ['1. "What is article 29 of the indian constitution about ?"\n', '2. "article 29 of indian constitution"\n', '3. "Explain article 29 of indian constitution in detail ?"']
INFO:langchain.retrievers.web_research:Searching for relevant urls...
INFO:langchain.retrievers.web_research:Searching for relevant urls...
INFO:langchain.retrievers.web_research:Search results: [{'title': 'Cultural and Educational Rights 

In [69]:
docs

[Document(page_content='**_To complement your preparation for the upcoming exam, check the following\nlinks:_**\n\n  *     * **Daily Video Analysis- The Hindu Newspaper**\n    * **Daily Press Information Bureau (PIB) Analysis**\n    * **Download NCERT Notes PDF (Free) for UPSC Exam**\n    * **Download Indian Polity Notes PDF (Free) for IAS Exam**\n    * **Latest Current Affairs**\n\n  \n---  \n  \n## Cultural and Educational Rights\n\nFundamental Rights guarantee basic rights to the citizens of India. There are\nsix fundamental rights enshrined in the Constitution of India, and Articles 29\nand 30 deals with the cultural and educational rights of Indian citizens.\n\n  1. This fundamental right intends to preserve the culture of minority groups in India.\n  2. Indian society is a composite heterogeneous one and its diversity is one of its strengths.\n  3. The Constitution guarantees these rights to minorities so that the diversity of this country is preserved and provides avenues for al

In [70]:
# Generate answer using retrieved docs
from langchain.chains.question_answering import load_qa_chain
chain = load_qa_chain(llm, chain_type="stuff")
output = chain({"input_documents": docs, "question": user_input},return_only_outputs=True)
output['output_text']

Your text contains a trailing whitespace, which has been trimmed to ensure high quality generations.


'Article 29 of the Indian Constitution deals with the protection of interests of minorities. It states that any section of citizens having a distinct language, script or culture have the right to conserve the same. It prohibits the State from discrimination against any citizen on grounds of religion, race, caste, language or any of them. \n\nThis Article also prohibits the State from denying admission to any citizen in institutions of education maintained by it or receiving aid from it. However, this Article does not deny the State from making any special provision for the educational advancement of any minority community or for the advancement of any tribe or caste or class of citizens or for the advancement of women or for the disadvantaged sections of the society. \n\nArticle 29 is in Part III of the Constitution, which is titled Fundamental Rights. Fundamental Rights are basic human rights that have been guaranteed by the Constitution of India to every citizen of India. These right

#### More Flexibility

- We can pass an LLM chain with custom prompt and output parsing

In [71]:
import os
import re
from typing import List
from langchain.chains import LLMChain
from pydantic import BaseModel, Field
from langchain.prompts import PromptTemplate
from langchain.output_parsers.pydantic import PydanticOutputParser

In [72]:
# LLMChain
search_prompt = PromptTemplate(
    input_variables=["question"],
    template="""You are an assistant tasked with improving Google search results limited to country INDIA only
    Generate FIVE Google search queries that are similar to this question. The output should be a numbered list of questions and each
    should have a question mark at the end: {question} """
)

In [73]:
class LineList(BaseModel):
    """List of questions."""
    lines: List[str] = Field(description="Questions")

class QuestionListOutputParser(PydanticOutputParser):
    """Output parser for a list of numbered questions."""
    def __init__(self) -> None:
        super().__init__(pydantic_object=LineList)

    def parse(self, text: str) -> LineList:
        lines = re.findall(r"\d+\..*?\n", text)
        return LineList(lines=lines)


In [74]:
llm_chain = LLMChain(
    llm=llm,
    prompt=search_prompt,
    output_parser = QuestionListOutputParser()
)

In [75]:
# Initialize
web_research_retriever_llm_chain = WebResearchRetriever(
    vectorstore=vectorstore,
    llm_chain=llm_chain,
    search=search
)

# Run
user_input = "What are writs?"
docs = web_research_retriever_llm_chain.get_relevant_documents(user_input)

INFO:langchain.retrievers.web_research:Generating questions for Google Search ...
Your text contains a trailing whitespace, which has been trimmed to ensure high quality generations.
INFO:langchain.retrievers.web_research:Questions for Google Search (raw): {'question': 'What are writs?', 'text': LineList(lines=['1. What are writs in India?\n', '2. Writs in the Indian Constitution\n', '3. Writs and their types\n', '4. List of writs in the Indian Constitution\n'])}
INFO:langchain.retrievers.web_research:Questions for Google Search: ['1. What are writs in India?\n', '2. Writs in the Indian Constitution\n', '3. Writs and their types\n', '4. List of writs in the Indian Constitution\n']
INFO:langchain.retrievers.web_research:Searching for relevant urls...
INFO:langchain.retrievers.web_research:Searching for relevant urls...
INFO:langchain.retrievers.web_research:Search results: [{'title': 'Types of Writs in Indian Constitution - Polity Notes', 'link': 'https://byjus.com/free-ias-prep/types-o

In [76]:
docs

[Document(page_content='## What is Writ?\n\nWrits are a written order from the Supreme Court or High Court that commands\nconstitutional remedies for Indian Citizens against the violation of their\nfundamental rights. Article 32 in the Indian Constitution deals with\nconstitutional remedies that an Indian citizen can seek from the Supreme Court\nof India and High Court against the violation of his/her fundamental rights.\nThe same article gives the Supreme Court power to issue writs for the\nenforcement of rights whereas the High Court has the same power under Article\n226. The writs- Habeas Corpus, Mandamus, Certiorari, Quo Warranto, and\nProhibition form an important topic for IAS Exam and its three stages-\nPrelims, Mains, and Interview.\n\n**Types of Writs in India - Indian Polity Notes**  \nDownload PDF Here\n\nThis article will mention the types of writs that come under the original\njurisdiction of the Supreme Court and High Courts, forming an important part\nof IAS Mains GS-II 

In [77]:
# Generate answer using retrieved docs
from langchain.chains.question_answering import load_qa_chain
chain = load_qa_chain(llm, chain_type="stuff")
output = chain({"input_documents": docs, "question": user_input},return_only_outputs=True)
output['output_text']

' A writ is a written order from a court. There are five types of writs: Habeas Corpus, Mandamus, Prohibition, Certiorari, and Quo Warranto.'